# Data Processing

This notebook demonstrates how to use `tgml` for common data processing tasks on graphs stored in `TigerGraph`.

## Define Graph

Conceptually, the `TigerGraph` class represents the graph stored in the database. Under the hood, it stores the necessary information to communicate with the TigerGraph database. It can read `username` and `password` from environment variables `TGUSERNAME` and `TGPASSWORD`. Hence, we recommend storing those credentials in the environment variables or in a `.env` file instead of hardcoding them in code. However, if you do provide `username` and `password` to this class constructor, the environment variables will be ignored.

In [1]:
from tgml.data import TigerGraph

Args to the `TigerGraph` class:
*    host (str, ): Address of the server. Defaults to "http://localhost".
*    graph (str, ): Name of the graph. Defaults to None.
*    username (str, optional): Username. Defaults to None.
*    password (str, optional): Password for the user. Defaults to None.
*    rest_port (str, optional): Port for the REST endpoint. Defaults to "9000".
*    gs_port (str, optional): Port for GraphStudio. Defaults to "14240".
*    token_auth (bool, optional): Whether to use token authentication. If True, token authentication must be turned on in the TigerGraph database server. Defaults to True.

In [2]:
tgraph = TigerGraph(host = "http://35.230.92.92",
                    graph = "Cora",
                    username = "tigergraph",
                    password = "tigergraphml")

In [3]:
tgraph.info()

Using graph 'Cora'
---- Graph Cora
Vertex Types: 
  - VERTEX Paper(PRIMARY_ID id INT, x LIST<INT>, y INT, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
Edge Types: 
  - DIRECTED EDGE Cite(FROM Paper, TO Paper)

Graphs: 
  - Graph Cora(Paper:v, Cite:e)
Jobs: 
  - CREATE LOADING JOB load_cora_data {
      DEFINE FILENAME edge_csv = "/home/tigergraph/data/Cora/edges.csv";
      DEFINE FILENAME node_csv = "/home/tigergraph/data/Cora/nodes.csv";
      LOAD node_csv TO VERTEX Paper VALUES($"id", SPLIT($"x", " "), $"y", $"train", $"valid", $"test") USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD edge_csv TO EDGE Cite VALUES($"source", $"target") USING SEPARATOR=",", HEADER="true", EOL="\n";
    }

Queries: 
  - export_vertex_train_mask_val_mask_test_mask(string output_path) (installed v2)
  - get_vertex_number(string v_type, string filter_by) (installed v2)
  - train_test_vertex_split(string train_attr, string test_

In [4]:
tgraph.number_of_vertices()

2708

In [5]:
tgraph.number_of_vertices("Paper")

2708

In [6]:
tgraph.number_of_edges()

10556

In [7]:
tgraph.number_of_edges("Cite")

10556

## Train/Validation/Test Split

In [13]:
from tgml.utils import split_vertices

`tgml` provide a utility function `split_vertices` to split vertices into a training, a validation, and a test set. More precisely, it creates 3 boolean attributes with each attribute indicating whether the vertex is in the corresponding set. For example, if you want to split the vertices into 80% train, 10% validation and 10% test, you can provide as arguments to the function `train_mask=0.8, val_mask=0.1, test_mask=0.1`. This will create 3 attributes `train_mask`, `val_mask`, `test_mask` in the graph, if they don't already exist. 80% of vertices will be set to `train_mask=1`, 10% to `val_mask=1`, and 10% to `test_mask=1` at random. There will be no overlap between the partitions. You can name the attributes however you like as long as you follow the format, such as `yesterday=0.8, today=0.1, tomorrow=0.1`, but we recommend something  meaningful. 

In [14]:
split_vertices(tgraph, train_mask=0.8, val_mask=0.1, test_mask=0.1)

Now the split is done. Load all vertices and check if the split is correct. See the next tutorial for details on `VertexLoader` and other data loaders.

In [15]:
from tgml.dataloaders import VertexLoader

In [16]:
%%time
vertex_loader = VertexLoader(tgraph, attributes="train_mask,val_mask,test_mask")

CPU times: user 6.39 ms, sys: 2.45 ms, total: 8.85 ms
Wall time: 24 s


In [17]:
%%time
data = vertex_loader.data

CPU times: user 79.2 ms, sys: 23.6 ms, total: 103 ms
Wall time: 16.8 s


In [18]:
data.train_mask.sum()/len(data), data.val_mask.sum()/len(data), data.test_mask.sum()/len(data)

(0.7996034615755472, 0.1000024105679298, 0.10039111464661074)

## Feature Engineering

We are adding graph algorithms to the workbench to perform feature engineering tasks. Current we are experimenting with PageRank and more details will be added shortly. For now you can use `pyTigerGraph` to install and run your own GSQL queries for feature extraction or any other task that can be done in GSQL.

In [11]:
import pyTigerGraph as tg

conn = tg.TigerGraphConnection(
    host="http://35.230.92.92",
    username="tigergraph",
    password="tigergraphml",
    graphname="Cora",
)

conn.apiToken = conn.getToken(conn.createSecret())

### PageRank

PageRank is the algorithm that originally ran Google's search engine, where it ranked the most influential webpages higher than less influential ones. A page was determined to be influential through its PageRank score, which is based off the importance of the pages that linked to it. Generalizing this, PageRank finds the most influential vertices in a graph based upon how influential the vertices that have edges to the desired vertex are.

The documentation of the PageRank query is found here: https://docs.tigergraph.com/graph-algorithm-library/centrality/pagerank.

#### Install PageRank Query

In [7]:
gsql = '''
CREATE QUERY tg_pagerank (STRING v_type, STRING e_type,
 FLOAT max_change=0.001, INT max_iter=25, FLOAT damping=0.85, INT top_k = 100,
 BOOL print_accum = TRUE, STRING result_attr =  "", STRING file_path = "",
 BOOL display_edges = FALSE) {
/*
 Compute the pageRank score for each vertex in the GRAPH
 In each iteration, compute a score for each vertex:
     score = (1-damping) + damping*sum(received scores FROM its neighbors).
 The pageRank algorithm stops when either of the following is true:
 a) it reaches max_iter iterations;
 b) the max score change for any vertex compared to the last iteration <= max_change.
 v_type: vertex types to traverse          print_accum: print JSON output
 e_type: edge types to traverse            result_attr: INT attr to store results to
 max_iter; max #iterations                 file_path: file to write CSV output to
 top_k: #top scores to output              display_edges: output edges for visualization
 max_change: max allowed change between iterations to achieve convergence
 damping: importance of traversal vs. random teleport

 This query supports only taking in a single edge for the time being (8/13/2020).
*/
	TYPEDEF TUPLE<VERTEX Vertex_ID, FLOAT score> Vertex_Score;
	HeapAccum<Vertex_Score>(top_k, score DESC) @@topScores;
	MaxAccum<FLOAT> @@max_diff = 9999;    # max score change in an iteration
	SumAccum<FLOAT> @recvd_score = 0; # sum of scores each vertex receives FROM neighbors
	SumAccum<FLOAT> @score = 1;           # initial score for every vertex is 1.
	SetAccum<EDGE> @@edgeSet;             # list of all edges, if display is needed
	FILE f (file_path);

# PageRank iterations	
	Start = {v_type};                     # Start with all vertices of specified type(s)
	WHILE @@max_diff > max_change LIMIT max_iter DO
			@@max_diff = 0;
			V = SELECT s
				FROM Start:s -(e_type:e)-> v_type:t
				ACCUM t.@recvd_score += s.@score/(s.outdegree(e_type)) 
				POST-ACCUM s.@score = (1.0-damping) + damping * s.@recvd_score,
						   s.@recvd_score = 0,
						   @@max_diff += abs(s.@score - s.@score');
	END; # END WHILE loop

# Output
	IF file_path != "" THEN
	  f.println("Vertex_ID", "PageRank");
	END;

	V = SELECT s FROM Start:s
		POST-ACCUM 
			IF result_attr != "" THEN s.setAttr(result_attr, s.@score) END,
			IF file_path != "" THEN f.println(s, s.@score) END,
			IF print_accum THEN @@topScores += Vertex_Score(s, s.@score) END;

	IF print_accum THEN
		PRINT @@topScores;
		IF display_edges THEN
			PRINT Start[Start.@score];
			Start = SELECT s
					FROM Start:s -(e_type:e)-> v_type:t
					ACCUM @@edgeSet += e;
		   PRINT @@edgeSet;
		END;
	END;
}
'''

In [8]:
# You can write any gsql query and install it with the following command.
print(conn.gsql(gsql + "\n INSTALL QUERY tg_pagerank"))

Successfully created queries: [tg_pagerank].
Start installing queries, about 1 minute ...
tg_pagerank query: curl -X GET 'http://127.0.0.1:9000/query/Cora/tg_pagerank?v_type=VALUE&e_type=VALUE&[max_change=VALUE]&[max_iter=VALUE]&[damping=VALUE]&[top_k=VALUE]&[print_accum=VALUE]&[result_attr=VALUE]&[file_path=VALUE]&[display_edges=VALUE]'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
Select 'm1' as compile server, now connecting ...
Node 'm1' is prepared as compile server.

Query installation finished.


#### Run PageRank Query

In [14]:
# Once your query is installed, you can run it using the 
# `runInstalledQuery` function with input parameters as a dict.

params = {
  "v_type": "Paper",
  "e_type": "Cite",
  "max_change": 0.001,
  "max_iter": 25,
  "damping": 0.85,
  "top_k": 150,
  "print_accum": True,
  "result_attr": "",
  "file_path": "",
  "display_edges": False
}

pr_res = conn.runInstalledQuery("tg_pagerank", params=params)

In [15]:
# Output from your query will be returned as a JSON
pr_res

[{'@@topScores': [{'Vertex_ID': '1358', 'score': 33.06402},
   {'Vertex_ID': '1701', 'score': 16.8922},
   {'Vertex_ID': '1986', 'score': 14.46646},
   {'Vertex_ID': '306', 'score': 13.72521},
   {'Vertex_ID': '1810', 'score': 9.81972},
   {'Vertex_ID': '2034', 'score': 8.61615},
   {'Vertex_ID': '1623', 'score': 7.57608},
   {'Vertex_ID': '88', 'score': 7.24722},
   {'Vertex_ID': '598', 'score': 7.13392},
   {'Vertex_ID': '1013', 'score': 6.85707},
   {'Vertex_ID': '1914', 'score': 6.5405},
   {'Vertex_ID': '1441', 'score': 6.47049},
   {'Vertex_ID': '733', 'score': 6.28126},
   {'Vertex_ID': '2045', 'score': 5.70577},
   {'Vertex_ID': '109', 'score': 5.70542},
   {'Vertex_ID': '1542', 'score': 5.68441},
   {'Vertex_ID': '1072', 'score': 5.67071},
   {'Vertex_ID': '1224', 'score': 5.1217},
   {'Vertex_ID': '1042', 'score': 4.88356},
   {'Vertex_ID': '1169', 'score': 4.53016},
   {'Vertex_ID': '415', 'score': 4.2736},
   {'Vertex_ID': '2182', 'score': 4.10047},
   {'Vertex_ID': '1692',